1) 데이터 가져오기

2) 데이터 중강

3) 데이터 전처리

# 0) 모듈 실행

##기본 모듈

In [1]:
# Matplotlib 와 pandas 그래프에서 한글 사용 설정
import sys

## 노트북이 코랩에서 실행중인지 체크
if 'google.colab' in sys.modules:
  !echo 'debconf debconf/frontend select Noninterative' | debconf-set-selections
  ## 나눔 폰트 설치
  !sudo apt-get -qq -y install fonts-nanum
  import matplotlib.font_manager as fm
  font_files = fm.findSystemFonts(fontpaths=['/usr/share/fonts/truetype/nanum'])
  for fpath in font_files:
    fm.fontManager.addfont(fpath)

  # 나눔바른고딕 폰트 설정
  import matplotlib.pyplot as plt
  plt.rc('font', family='NanumBarunGothic')
  # 마이너스 기호 표시 오류 수정
  import matplotlib
  matplotlib.rcParams['axes.unicode_minus'] = False


In [2]:
!pip install konlpy

In [3]:
!apt-get update -qq

In [4]:
!apt-get install fonts-nanum*  -qq

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import random
import os

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## KoBERT 설치
- 서비스 작동 중지로 인해 hugging face를 통해 모델 다운로드 전환
- https://complexoftaste.tistory.com/2

In [6]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

In [7]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-b9mj6gts/kobert-tokenizer_7d37aeddbe064fa7b5c6eeb95bbbae88
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-b9mj6gts/kobert-tokenizer_7d37aeddbe064fa7b5c6eeb95bbbae88
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [8]:
!pip3 install mxnet-mkl==1.6.0 numpy==1.23.1

In [9]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [10]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [11]:
# GPU 사용 시
device = torch.device("cuda:0")

In [12]:
# ★
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

# 1) 데이터 가져오기

## 감성대화 말뭉치
https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=86
- 일반인 1,500명 대상의 인터뷰 및 크라우드 소싱 수행
- 우울증 환자 대상 WOZ 대화 수집

=> 전처리 과정으로 "한사람만의 대화를 줄글로 변형 - 질문 제거"

### 1. 데이터 가져오기

In [13]:
train = pd.read_excel("/content/drive/MyDrive/0000/Class/졸프/Dataset/AIhub_감성대화/Training_221115_add/원천데이터/감성대화말뭉치(최종데이터)_Training.xlsx")
valid = pd.read_excel("/content/drive/MyDrive/0000/Class/졸프/Dataset/AIhub_감성대화/Validation_221115_add/원천데이터/감성대화말뭉치(최종데이터)_Validation.xlsx")

In [14]:
train_df = train.copy()
valid_df = valid.copy()

### 2. 전처리

In [15]:
# "사람문장3" 결측값 대체
train_df.fillna('',inplace=True)
valid_df.fillna('',inplace=True)

# 문장 병합
train_df['사람문장'] = train_df['사람문장1']+train_df['사람문장2']+train_df['사람문장3']
valid_df['사람문장'] = valid_df['사람문장1']+valid_df['사람문장2']+valid_df['사람문장3']

# "감정 분류"과 "문장"만 추출
train_df = train_df[['사람문장', '감정_대분류']]
valid_df = valid_df[['사람문장', '감정_대분류']]

# 컬럼명 변경
train_df.rename(columns={'사람문장': 'text',
                   '감정_대분류': 'sentiment'}, inplace=True)
valid_df.rename(columns={'사람문장': 'text',
                   '감정_대분류': 'sentiment'}, inplace=True)

# label encoding
le = LabelEncoder()
le=le.fit(train_df['sentiment'])
train_df['target']=le.transform(train_df['sentiment'])

le=le.fit(valid_df['sentiment'])
valid_df['target']=le.transform(valid_df['sentiment'])

# text, target
train_conv = train_df[['text','target']]
valid_conv = valid_df[['text','target']]

In [16]:
# 정규표현식으로 한국어만 남기기
import re
from tqdm import tqdm

def removing_non_korean(df):
    for idx, row in tqdm(df.iterrows(), desc='removing_non_korean', total=len(df)):
        new_doc = re.sub('[^가-힣]', ' ', row['text']).strip() # 특수문자 제거
        df.loc[idx, 'document'] = new_doc
    return df

# 적용
train_conv['text'] = removing_non_korean(train_conv[['text']])['document']
valid_conv['text'] = removing_non_korean(valid_conv[['text']])['document']

removing_non_korean: 100%|██████████| 51630/51630 [00:08<00:00, 5939.89it/s]
<ipython-input-16-c3101e0deaa4>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_conv['text'] = removing_non_korean(train_conv[['text']])['document']
removing_non_korean: 100%|██████████| 6641/6641 [00:01<00:00, 4811.71it/s]
<ipython-input-16-c3101e0deaa4>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_conv['text'] = removing_non_korean(valid_conv[['text']])['document']


In [17]:
# 라벨 인코딩 결과 확인
sentiment = train_df[['sentiment', 'target']].drop_duplicates(subset=['sentiment']) # sentiment 컬럼에 대해 중복된 값을 갖고 있는 열 제거
df_unique = sentiment.sort_values(by='target', ascending=True)
df_unique = df_unique.reset_index(drop=True)
df_unique

,sentiment,target
0,기쁨,0
1,당황,1
2,분노,2
3,불안,3
4,상처,4
5,슬픔,5


# 2) 데이터 증강
데이터 불균형으로 인해 분류 전에 전처리하는 방식인 smote 택함

**SMOTE(= Synthetic Minority Over-sampling Technique)**

SMOTE는 합성을 기반으로 하는 방법으로, minority class의 샘플을 가져와 이들을 잇는 선에서 중간값을 택하여 만들어내는 방식

Train

In [18]:
# SMOTE : 오버샘플링
from imblearn.over_sampling import RandomOverSampler
text = train_conv.text.to_numpy().reshape(-1, 1)
target = train_conv.target.to_numpy().reshape(-1, 1)

oversample = RandomOverSampler()

X_oversample, y_oversample = oversample.fit_resample(text, target)
train_conv2 = pd.DataFrame({"text": X_oversample.reshape(-1), 'target': y_oversample.reshape(-1)})
train_conv2

,text,target
0,일은 왜 해도 해도 끝이 없을까 화가 난다 그냥 내가 해결하는 게 나아 남들한테...,2
1,이번 달에 또 급여가 깎였어 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나 ...,2
2,회사에 신입이 들어왔는데 말투가 거슬려 그런 애를 매일 봐야 한다고 생각하니까 스...,2
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜 일도 많은 데 정말 분하고 ...,2
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나 상사인 나에게 먼...,2
...,...,...
55915,나 정말 속상해 이번에 수리 엄청 열심히 준비했는데 점수가 더 떨어졌어 애써도 안...,5
55916,중년 우울증으로 가족들에게 예민하고 조그만 갈등에도 화를 내 이런 상황이 슬퍼 예...,5
55917,자식들만 기다리며 혼자 사는 삶이 적막해 그때마다 슬픔을 느껴 자식들도 모두 출가...,5
55918,친한 친구와 애들처럼 몸싸움을 해서 당황스러워 정말 사소한 말다툼이었는데 몸싸움으로...,5


valid

In [19]:
# SMOTE : 오버샘플링
from imblearn.over_sampling import RandomOverSampler
text = valid_conv.text.to_numpy().reshape(-1, 1)
target = valid_conv.target.to_numpy().reshape(-1, 1)

oversample = RandomOverSampler()

X_oversample, y_oversample = oversample.fit_resample(text, target)
valid_conv2 = pd.DataFrame({"text": X_oversample.reshape(-1), 'target': y_oversample.reshape(-1)})
valid_conv2

,text,target
0,이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어 너...,3
1,회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워 나에게 너무 크게...,3
2,상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워 무섭게 생겼는데도 업무를 보려면...,3
3,이번에 힘들게 들어간 첫 직장이거든 첫 직장이라서 그런지 너무 긴장된다 첫 직장이...,3
4,직장에서 동료들이랑 관계가 안 좋아질까 봐 걱정돼 내가 낯가림이 심해서 친해질 수 ...,3
...,...,...
7537,의도하지 않았지만 친구에게 상처를 준 것 같아서 후회가 돼 친구의 눈이 작아 단춧구...,5
7538,지방으로 발령을 받았어 동기 중에는 내가 처음이야 조금 슬프네 다음 달에 가야 ...,5
7539,친구들이 괴롭히는 게 다 내 탓 같아 그래서 슬프다 앞으로도 친구를 사귀지 못할 ...,5
7540,마지막 이직을 생각하고 이 회사로 왔는데 재무 구조가 부실해서 후회하는 중이야 이렇...,5


In [20]:
# 데이터셋이 잘 나눠져있는지 확인
print(len(train_conv2))
print(len(valid_conv2))

55920
7542


# 3) 데이터 전처리
모델 적합을 위해 데이터가 적절히 전처리되어야 하며 이때 전처리는 토큰화, 정수 인코딩, 패딩을 의미함

- https://velog.io/@seolini43/KOBERT%EB%A1%9C-%EB%8B%A4%EC%A4%91-%EB%B6%84%EB%A5%98-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0-%ED%8C%8C%EC%9D%B4%EC%8D%ACColab

BERT는 이미 누군가가 학습해둔 모델을 사용하기 때문에 모델과 토크나이저가 항상 매핑되는 관계여야 함
- https://hoit1302.tistory.com/159
- https://github.com/SKTBrain/KoBERT/blob/master/kobert_hf/kobert_tokenizer/kobert_tokenizer.py

In [21]:
dataset_train = train_conv2.copy()
dataset_test = valid_conv2.copy()

In [22]:
dataset_train

,text,target
0,일은 왜 해도 해도 끝이 없을까 화가 난다 그냥 내가 해결하는 게 나아 남들한테...,2
1,이번 달에 또 급여가 깎였어 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나 ...,2
2,회사에 신입이 들어왔는데 말투가 거슬려 그런 애를 매일 봐야 한다고 생각하니까 스...,2
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜 일도 많은 데 정말 분하고 ...,2
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나 상사인 나에게 먼...,2
...,...,...
55915,나 정말 속상해 이번에 수리 엄청 열심히 준비했는데 점수가 더 떨어졌어 애써도 안...,5
55916,중년 우울증으로 가족들에게 예민하고 조그만 갈등에도 화를 내 이런 상황이 슬퍼 예...,5
55917,자식들만 기다리며 혼자 사는 삶이 적막해 그때마다 슬픔을 느껴 자식들도 모두 출가...,5
55918,친한 친구와 애들처럼 몸싸움을 해서 당황스러워 정말 사소한 말다툼이었는데 몸싸움으로...,5


In [23]:
dataset_test

,text,target
0,이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어 너...,3
1,회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워 나에게 너무 크게...,3
2,상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워 무섭게 생겼는데도 업무를 보려면...,3
3,이번에 힘들게 들어간 첫 직장이거든 첫 직장이라서 그런지 너무 긴장된다 첫 직장이...,3
4,직장에서 동료들이랑 관계가 안 좋아질까 봐 걱정돼 내가 낯가림이 심해서 친해질 수 ...,3
...,...,...
7537,의도하지 않았지만 친구에게 상처를 준 것 같아서 후회가 돼 친구의 눈이 작아 단춧구...,5
7538,지방으로 발령을 받았어 동기 중에는 내가 처음이야 조금 슬프네 다음 달에 가야 ...,5
7539,친구들이 괴롭히는 게 다 내 탓 같아 그래서 슬프다 앞으로도 친구를 사귀지 못할 ...,5
7540,마지막 이직을 생각하고 이 회사로 왔는데 재무 구조가 부실해서 후회하는 중이야 이렇...,5


In [24]:
train_data_list = []
for label, q  in zip(dataset_train['target'], dataset_train['text'])  :
    data = []
    data.append(q)
    data.append(str(label))

    train_data_list.append(data)

#train_data_list

test_data_list = []
for label, q in zip(dataset_test['target'], dataset_test['text'])  :
    data = []
    data.append(q)
    data.append(str(label))

    test_data_list.append(data)

#test_data_list

dataset_train = train_data_list
dataset_test = test_data_list

- 문장을 토큰화하고 특수 토큰([CLS], [SEP])을 추가
- 토큰을 정수 인덱스로 변환하고 패딩을 추가
- 옵션에 따라 문장 페어를 처리(문장 페어가 있는 경우 [SEP] 토큰을 사용하여 두 번째 문장을 추가합니다).

In [25]:
text = dataset_train[0]
print(text)

# KoBERTTokenizer를 사용하여 토큰화
tokenized = tokenizer(text[0], return_tensors='pt')
print(tokenized)

# 토큰화된 입력을 평평하게 만듦
flat_input_ids = tokenized['input_ids'].view(-1)
print(flat_input_ids)

# 토큰 ID를 단어로 변환하여 출력
decoded = tokenizer.decode(flat_input_ids)
print(decoded.split())

['일은 왜 해도 해도 끝이 없을까  화가 난다 그냥 내가 해결하는 게 나아  남들한테 부담 주고 싶지도 않고', '2']
{'input_ids': tensor([[   2, 3803, 7086, 3466, 5002, 5002, 1363, 7096, 3282, 5591, 5112, 5330,
         1406, 5782, 1189, 1435, 4999, 7794,  921, 1370, 6797, 1409, 5931, 7835,
         2426, 4217,  517, 6751, 7328, 3149,    3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])}
tensor([   2, 3803, 7086, 3466, 5002, 5002, 1363, 7096, 3282, 5591, 5112, 5330,
        1406, 5782, 1189, 1435, 4999, 7794,  921, 1370, 6797, 1409, 5931, 7835,
        2426, 4217,  517, 6751, 7328, 3149,    3])
['[CLS]일은', '왜', '해도', '해도', '끝이', '없을까', '화가', '난다', '그냥', '내가', '해결하는', '게', '나아', '남들한테', '부담', '주고', '싶지도', '않고[SEP]']


In [26]:
# Setting parameters - base
max_len = 26
batch_size = 10 # GPU 메모리가 너무 작아서
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [27]:
class BERTSentenceTransform:

    def __init__(self, tokenizer, max_seq_length, vocab=None, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = self._tokenizer.vocab if vocab is None else vocab
        # RoBERTa does not register CLS token and SEP token
        # 문장을 토큰화하고 특수 토큰([CLS], [SEP])을 추가
        if hasattr(self._vocab, 'cls_token'):
            self._cls_token = self._vocab.cls_token
        else:
            self._cls_token = self._vocab.bos_token
        if hasattr(self._vocab, 'sep_token'):
            self._sep_token = self._vocab.sep_token
        else:
            self._sep_token = self._vocab.eos_token
        self._padding_token = self._vocab.padding_token

    def __call__(self, line):
        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer.tokenize(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = []
        tokens.append(self._cls_token)
        tokens.extend(tokens_a)
        tokens.append(self._sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(self._sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._vocab[tokens]

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)
# #토큰을 정수 인덱스로 변환하고 패딩을 추가
        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([self._vocab[self._padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'), np.array(segment_ids, dtype='int32')


    def _truncate_seq_pair(self, tokens_a, tokens_b, max_length):
        """Truncates a sequence pair in place to the maximum length."""
        # This is a simple heuristic which will always truncate the longer sequence
        # one token at a time. This makes more sense than truncating an equal percent
        # of tokens from each, since if one sequence is very short then each token
        # that's truncated likely contains more information than a longer sequence.
        while True:
            total_length = len(tokens_a) + len(tokens_b)
            if total_length <= max_length:
                break
            if len(tokens_a) > len(tokens_b):
                tokens_a.pop()
            else:
                tokens_b.pop()


In [28]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer,
                                          max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [29]:
#토큰화
tok = tokenizer.tokenize

data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)

In [31]:
# torch 형식의 dataset을 만들어 입력 데이터셋의 전처리 마무리
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)

# 4) Kobert 모델 구현
- https://velog.io/@jaylnne/Pytorch-Lightning-%EC%9C%BC%EB%A1%9C-koBERT-Fine-Tuning-%ED%95%B4%EB%B3%B4%EA%B8%B0-NSMC

In [32]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 7,   # 감정 클래스 수로 조정
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [35]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


학습

In [36]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-36-c43874aa7363>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/5592 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.9150264263153076 train acc 0.4
epoch 1 batch id 201 loss 1.8395049571990967 train acc 0.1721393034825872
epoch 1 batch id 401 loss 1.8345539569854736 train acc 0.19775561097256864
epoch 1 batch id 601 loss 1.689703345298767 train acc 0.1993344425956736
epoch 1 batch id 801 loss 1.967494010925293 train acc 0.23270911360798996
epoch 1 batch id 1001 loss 1.4848963022232056 train acc 0.2856143856143854
epoch 1 batch id 1201 loss 1.2003953456878662 train acc 0.32006661115736895
epoch 1 batch id 1401 loss 2.010901689529419 train acc 0.3473947180585297
epoch 1 batch id 1601 loss 3.2014355659484863 train acc 0.374640849469082
epoch 1 batch id 1801 loss 1.6252415180206299 train acc 0.3868961687951141
epoch 1 batch id 2001 loss 2.639375686645508 train acc 0.39985007496251895
epoch 1 batch id 2201 loss 1.3088185787200928 train acc 0.41258518855065907
epoch 1 batch id 2401 loss 2.7847800254821777 train acc 0.4291545189504378
epoch 1 batch id 2601 loss 1.0294082164764404 t

<ipython-input-36-c43874aa7363>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/755 [00:00<?, ?it/s]

...

epoch 4 test acc 0.16649006622516546


  0%|          | 0/5592 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 2.8979175090789795 train acc 0.2
epoch 5 batch id 201 loss 1.8485229015350342 train acc 0.18258706467661703
epoch 5 batch id 401 loss 1.7793223857879639 train acc 0.19251870324189527
epoch 5 batch id 601 loss 1.743726372718811 train acc 0.1895174708818632
epoch 5 batch id 801 loss 1.6884183883666992 train acc 0.18826466916354476
epoch 5 batch id 1001 loss 1.828565239906311 train acc 0.18551448551448443
epoch 5 batch id 1201 loss 1.6702197790145874 train acc 0.18534554537884954
epoch 5 batch id 1401 loss 1.801346778869629 train acc 0.18486795146323934
epoch 5 batch id 1601 loss 3.038757801055908 train acc 0.18888194878201048
epoch 5 batch id 1801 loss 1.7058604955673218 train acc 0.19439200444197619
epoch 5 batch id 2001 loss 1.6211788654327393 train acc 0.2030484757621187
epoch 5 batch id 2201 loss 1.5607246160507202 train acc 0.20772376192639702
epoch 5 batch id 2401 loss 1.9763383865356445 train acc 0.2154518950437318
epoch 5 batch id 2601 loss 1.6404088735580

  0%|          | 0/755 [00:00<?, ?it/s]

epoch 5 test acc 0.34582781456953654


In [37]:
torch.save(model, 'model_KoBert.pt')

# 5) 테스트

In [40]:
def predict(predict_sentence): # input = 감정분류하고자 하는 sentence

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False) # 토큰화한 문장
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size = batch_size, num_workers = 5) # torch 형식 변환

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length = valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval = []
        for i in out: # out = model(token_ids, valid_length, segment_ids)
            logits = i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("기쁨")
            elif np.argmax(logits) == 1:
                test_eval.append("당황")
            elif np.argmax(logits) == 2:
                test_eval.append("분노")
            elif np.argmax(logits) == 3:
                test_eval.append("뷸언")
            elif np.argmax(logits) == 4:
                test_eval.append("상처")
            elif np.argmax(logits) == 5:
                test_eval.append("슬픔")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [39]:
# 질문에 0 입력 시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 집에 가고 싶다


AttributeError: 'function' object has no attribute 'tokenize'